In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('whitegrid')
import os
import re

os.chdir('/home/anovax/open_data/edrsr_data')

In [35]:
all_df = pd.read_csv('/home/anovax/open_data/edrsr_data/2019/documents.csv', '\t')

In [7]:
all_df.head()

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
0,78923438,2034,5,2.0,NaN,635/4979/18,2019-01-01 00:00:00+02,2019-01-01 00:00:00+02,Савченко Д. М.,NaN,0,2019-01-01 00:00:00+02
1,78923390,1604,5,2.0,NaN,527/1/19,2019-01-01 00:00:00+02,2019-01-01 00:00:00+02,Левицька Т. В.,NaN,0,2019-01-01 00:00:00+02
2,78923385,1604,5,2.0,NaN,527/1/19,2019-01-01 00:00:00+02,2019-01-01 00:00:00+02,Левицька Т. В.,NaN,0,2019-01-01 00:00:00+02
3,78923393,1716,2,5.0,6062.0,570/480/17,2018-12-07 00:00:00+02,2019-01-01 00:00:00+02,Остапчук Л.В.,http://od.reyestr.court.gov.ua/files/39/e775b7...,1,2019-01-02 00:00:00+02
4,78923435,915,3,1.0,1060.0,352/2383/18,2018-12-26 00:00:00+02,2019-01-01 00:00:00+02,Хоминець М. М.,http://od.reyestr.court.gov.ua/files/39/30b091...,1,2019-01-02 00:00:00+02


In [21]:
categories = pd.read_csv('/home/anovax/open_data/edrsr_data/2019/cause_categories.csv', '\t').rename({'name': 'category_name'},
                                                                                                    axis=1)

In [20]:
j_forms = pd.read_csv('/home/anovax/open_data/edrsr_data/2019/judgment_forms.csv', '\t').rename({'name': 'j_form_name'},
                                                                                                    axis=1)

j_kinds = pd.read_csv('/home/anovax/open_data/edrsr_data/2019/justice_kinds.csv', '\t').rename({'name': 'j_kind_name'},
                                                                                                    axis=1)

In [36]:
merged = pd.merge(all_df, categories)

In [37]:
merged = pd.merge(merged, j_forms)

In [38]:
merged = pd.merge(merged, j_kinds)

In [39]:
merged.shape

(1618897, 15)

In [17]:
merged.head()

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ,category_name,j_form_name,j_kind_name
0,78923393,1716,2,5.0,6062.0,570/480/17,2018-12-07 00:00:00+02,2019-01-01 00:00:00+02,Остапчук Л.В.,http://od.reyestr.court.gov.ua/files/39/e775b7...,1,2019-01-02 00:00:00+02,"Вчинення домашнього насильства, насильства за ...",Постанова,Адмінправопорушення
1,78923425,1716,2,5.0,6062.0,570/4649/17,2018-12-07 00:00:00+02,2019-01-01 00:00:00+02,Остапчук Л.В.,http://od.reyestr.court.gov.ua/files/39/acea1e...,1,2019-01-02 00:00:00+02,"Вчинення домашнього насильства, насильства за ...",Постанова,Адмінправопорушення
2,78923377,1716,2,5.0,6062.0,570/4707/17,2018-12-07 00:00:00+02,2019-01-01 00:00:00+02,Остапчук Л.В.,http://od.reyestr.court.gov.ua/files/39/f22939...,1,2019-01-02 00:00:00+02,"Вчинення домашнього насильства, насильства за ...",Постанова,Адмінправопорушення
3,78923424,1716,2,5.0,6062.0,570/4607/17,2018-12-07 00:00:00+02,2019-01-01 00:00:00+02,Остапчук Л.В.,http://od.reyestr.court.gov.ua/files/39/b3974d...,1,2019-01-02 00:00:00+02,"Вчинення домашнього насильства, насильства за ...",Постанова,Адмінправопорушення
4,78938570,1506,2,5.0,6062.0,496/5358/18,2018-12-26 00:00:00+02,2019-01-02 00:00:00+02,Горяєв І. М.,http://od.reyestr.court.gov.ua/files/39/6a87f7...,1,2019-01-03 00:00:00+02,"Вчинення домашнього насильства, насильства за ...",Постанова,Адмінправопорушення


In [40]:
family_mask = (merged.category_name =='Спори, що виникають із сімейних правовідносин про розірвання шлюбу') | \
                (merged.category_name =='про розірвання шлюбу') | \
                 (merged.category_name =='Спори, що виникають із сімейних правовідносин про стягнення аліментів')

In [41]:
family_cases_2019 = merged.loc[family_mask]

In [42]:
family_cases_2019.shape

(98407, 15)

In [43]:
decisions_mask = (family_cases_2019.j_form_name == 'Рішення') | (family_cases_2019.j_form_name == 'Постанова')

In [45]:
family_cases_2019 = family_cases_2019[decisions_mask]

In [13]:
def get_df_from_chunker(path):
    chunker = pd.read_csv(path, '\t', chunksize=10_000)
    dec_df = pd.DataFrame()

    for piece in chunker:
        
        
        
        piece = piece.loc[(piece.justice_kind == 1) & \
                   ((piece.judgment_code == 2) | (piece.judgment_code == 3))]
        
        
        if dec_df.empty:
            
            dec_df = piece
        else:
            dec_df = pd.concat([dec_df, piece], axis=0)
            

    
    return dec_df

In [14]:
civil_cases_2018 = get_df_from_chunker('/home/anovax/open_data/edrsr_data/2018/documents.csv')

(1243, 12)
(2599, 12)
(3880, 12)
(5224, 12)
(6350, 12)
(7505, 12)
(8667, 12)
(9512, 12)
(10539, 12)
(11073, 12)
(11930, 12)
(12819, 12)
(13609, 12)
(14289, 12)
(14862, 12)
(15408, 12)
(16004, 12)
(16922, 12)
(17703, 12)
(18487, 12)
(19303, 12)
(20080, 12)
(21219, 12)
(21959, 12)
(22954, 12)
(23688, 12)
(24518, 12)
(25477, 12)
(26845, 12)
(27834, 12)
(28931, 12)
(30007, 12)
(30947, 12)
(31770, 12)
(33076, 12)
(34369, 12)
(34959, 12)
(35496, 12)
(36457, 12)
(37328, 12)
(38192, 12)
(39357, 12)
(40231, 12)
(41102, 12)
(42489, 12)
(43494, 12)
(44467, 12)
(45333, 12)
(46452, 12)
(47407, 12)
(48488, 12)
(49507, 12)
(50697, 12)
(51444, 12)
(52240, 12)
(53455, 12)
(54273, 12)
(55200, 12)
(56489, 12)
(57481, 12)
(58499, 12)
(59501, 12)
(60539, 12)
(61582, 12)
(62546, 12)
(63370, 12)
(64136, 12)
(65203, 12)
(66160, 12)
(67149, 12)
(68089, 12)
(69210, 12)
(69883, 12)
(71004, 12)
(72284, 12)
(73404, 12)
(74569, 12)
(75705, 12)
(76820, 12)
(78103, 12)
(79003, 12)
(79911, 12)
(81474, 12)
(82504, 12)


(671275, 12)
(672584, 12)
(673673, 12)
(674737, 12)
(676089, 12)
(677310, 12)
(678492, 12)
(679517, 12)
(680749, 12)
(681733, 12)
(682622, 12)
(684109, 12)
(685173, 12)
(686210, 12)
(687699, 12)
(688824, 12)
(689825, 12)
(691270, 12)
(692667, 12)
(693868, 12)
(695001, 12)
(695901, 12)
(697130, 12)
(698232, 12)
(699476, 12)
(700423, 12)
(701364, 12)
(702649, 12)
(704166, 12)
(705238, 12)
(706349, 12)
(707927, 12)
(708990, 12)
(710065, 12)
(711950, 12)
(713346, 12)
(714444, 12)
(715495, 12)
(716466, 12)
(717638, 12)
(719154, 12)
(720391, 12)
(721543, 12)
(722688, 12)
(723812, 12)
(724916, 12)
(726220, 12)
(727392, 12)
(728494, 12)
(729566, 12)
(731337, 12)
(732339, 12)
(733576, 12)
(735182, 12)
(735664, 12)
(736847, 12)
(738041, 12)
(740002, 12)
(741177, 12)
(742295, 12)
(743230, 12)
(744637, 12)
(745550, 12)
(746561, 12)
(748223, 12)
(749300, 12)
(750385, 12)
(751414, 12)
(753245, 12)
(754289, 12)
(755413, 12)
(756499, 12)
(757661, 12)
(759316, 12)
(760388, 12)
(761508, 12)
(762482, 12)

In [23]:
family_mask = (categories.category_name =='Спори, що виникають із сімейних правовідносин про розірвання шлюбу') | \
                (categories.category_name =='про розірвання шлюбу') | \
                 (categories.category_name =='Спори, що виникають із сімейних правовідносин про стягнення аліментів')

In [24]:
categories[family_mask]

,category_code,category_name
152,1078,"Спори, що виникають із сімейних правовідносин ..."
1513,1079,"Спори, що виникають із сімейних правовідносин ..."
2231,40376,про розірвання шлюбу


In [27]:
# codes for whole dataframe

family_mask = (civil_cases_2018.category_code == 1078) | \
                (civil_cases_2018.category_code == 1079) | \
                 (civil_cases_2018.category_code == 40376)

In [28]:
family_cases_2018 = civil_cases_2018[family_mask]

In [34]:
family_cases_2018.shape

(182001, 12)

In [49]:
family_cases_2018 = pd.merge(family_cases_2018, categories)

NameError: name 'merged_2018' is not defined

In [50]:
family_cases_2018 = pd.merge(family_cases_2018, j_forms)

family_cases_2018 = pd.merge(family_cases_2018, j_kinds)

In [51]:
family_cases_2018.shape

(182001, 15)

In [55]:
family_cases_2018.dropna(subset=['doc_url'], inplace=True)

In [56]:
family_cases_2018.shape

(179818, 15)

In [60]:
family_cases_2019.dropna(subset=['doc_url'], inplace=True)

In [61]:
family_cases_2019.shape

(35659, 15)

In [52]:
family_cases_2018.head()

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ,category_name,j_form_name,j_kind_name
0,71388659,2609,3.0,1.0,1078.0,760/25440/17,2017-12-19 00:00:00+02,2018-01-02 00:00:00+02,Кушнір С. І.,http://od.reyestr.court.gov.ua/files/35/0ace93...,1,2018-01-03 00:00:00+02,"Спори, що виникають із сімейних правовідносин ...",Рішення,Цивільне
1,71378040,401,3.0,1.0,1078.0,199/6837/17,2017-12-29 00:00:00+02,2018-01-02 00:00:00+02,Скрипник О. Г.,NaN,0,2018-01-03 00:00:00+02,"Спори, що виникають із сімейних правовідносин ...",Рішення,Цивільне
2,71367436,2606,3.0,1.0,1078.0,757/42636/17-ц,2017-12-13 00:00:00+02,2018-01-02 00:00:00+02,Москаленко К. О.,http://od.reyestr.court.gov.ua/files/35/71959a...,1,2018-01-03 00:00:00+02,"Спори, що виникають із сімейних правовідносин ...",Рішення,Цивільне
3,71377948,401,3.0,1.0,1078.0,199/4892/17,2017-12-28 00:00:00+02,2018-01-02 00:00:00+02,Скрипник О. Г.,NaN,0,2018-01-03 00:00:00+02,"Спори, що виникають із сімейних правовідносин ...",Рішення,Цивільне
4,71368581,2610,3.0,1.0,1078.0,761/30956/17,2017-12-15 00:00:00+02,2018-01-02 00:00:00+02,Макаренко І. О.,http://od.reyestr.court.gov.ua/files/35/301fe6...,1,2018-01-03 00:00:00+02,"Спори, що виникають із сімейних правовідносин ...",Рішення,Цивільне


In [62]:
family_cases_1819 = family_cases_2018.append(family_cases_2019)

In [63]:
family_cases_1819.shape

(215477, 15)

In [68]:
(family_cases_1819.category_name == 'Спори, що виникають із сімейних правовідносин про стягнення аліментів').sum()

81692

In [69]:
family_cases_1819.to_csv('family_cases_1819.csv', index=False)

AttributeError: 'DataFrame' object has no attribute 'id'

In [71]:
family_cases_1819.doc_id.to_csv('family_cases_ids.csv', index=False)

## Getting durations in days

In [ ]:
def get_year_range_df(year_range):
    os.chdir('/home/anovax/open_data/edrsr_data')

    df_list = []

    for year in year_range:
        path = '{}/documents.csv'.format(year, year)
        df = get_df_from_chunker(path).drop(['status', 'date_publ', 'receipt_date'], axis=1)
        df_list.append(df)
    
    merged_df = pd.concat(df_list, axis=0, ignore_index=True)
    
    return merged_df

In [ ]:
family_cases = pd.read_csv('family_cases_1819.csv')

In [ ]:
family_cause_nums = family_cases.cause_num.tolist()

In [ ]:
courts = pd.read_csv('/home/anovax/open_data/edrsr_data/2019/courts.csv', '\t')
instances = pd.read_csv('/home/anovax/open_data/edrsr_data/2019/instances.csv', '\t')

In [ ]:
def get_df_from_chunker(path):
    chunker = pd.read_csv(path, '\t', chunksize=10000)
    dec_df = pd.DataFrame()

    for piece in chunker:
        piece = piece[piece.cause_num.isin(family_cause_nums)]
        print(piece.shape)
        
        if dec_df.empty:
            dec_df = piece
        else:
            dec_df = pd.concat([dec_df, piece], axis=0)
    
    return dec_df

In [ ]:
year_range = list(range(2014, 2019+1))

fc_1419 = get_year_range_df(year_range)

In [ ]:
fc_1419.to_csv('fc_1419_for_duration.csv', index=False)

In [ ]:
fc_1419_more = pd.merge(pd.merge(fc_1419, courts), instances, on='instance_code')

In [ ]:
days_duration_diff = fc_1419_more.groupby(['cause_num', 'name_y']).adjudication_date.apply(lambda x: x.apply(pd.to_datetime).max() 
                                                        - x.apply(pd.to_datetime).min())

In [ ]:
duration_df = days_duration_diff.reset_index()

In [ ]:
duration_df.to_csv('family_cases_duration.csv', index=False)